Nama: Bernard Hugo

NIM: 2540124450

Kelas: LA05

Assignment 2 Text Mining

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import dan read dataset csv

In [ ]:
import pandas as pd

df = pd.read_csv('drive/MyDrive/train_data.csv')
df

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0
...,...,...
16985,KfW credit line for Uniper could be raised to ...,3
16986,KfW credit line for Uniper could be raised to ...,3
16987,Russian https://t.co/R0iPhyo5p7 sells 1 bln r...,3
16988,Global ESG bond issuance posts H1 dip as supra...,3


In [ ]:
label = df["label"].unique()

for i in label:
    print(i)

0
2
9
8
4
5
6
1
7
10
11
12
13
14
15
17
16
18
19
3


Melakukan preprocessing pada data

In [ ]:
# Preprocessing
import re

# case folding
df['clean_text'] = df['text'].str.lower()
# remove &amp (&)
df['clean_text'] = [i.replace("&amp;", "") for i in df['clean_text']]
# remove url
df['clean_text'] = [re.sub(r'https?:\/\/(?:www\.)?[-\w@:%.\+~#=]{1,256}\.[a-zA-Z\d()]{1,6}\b(?:[-\w()@:%\+.~#?&//=]*)','',i) for i in df['clean_text']]
# remove whitespace
df['clean_text'] = [re.sub(r'\s+',' ',i) for i in df['clean_text']]
# remove number
df['clean_text'] = [re.sub(r'\d+','',i) for i in df['clean_text']]
# remove non-alpha numeric characters
df['clean_text'] = [re.sub(r'[^\w\s]','',i) for i in df['clean_text']]

df['clean_text']

0        here are thursdays biggest analyst calls apple...
1        buy las vegas sands as travel to singapore bui...
2        piper sandler downgrades docusign to sell citi...
3        analysts react to teslas latest earnings break...
4        netflix and its peers are set for a return to ...
                               ...                        
16985    kfw credit line for uniper could be raised to ...
16986    kfw credit line for uniper could be raised to ...
16987    russian sells  bln roubles at oneyear repo auc...
16988    global esg bond issuance posts h dip as supran...
16989    brazils petrobras says it signed a  billion su...
Name: clean_text, Length: 16990, dtype: object

Hasil yang didapatkan adalah data-data teks yang sudah dibersihkan dimana tanda baca, angka, dan whitespace yang berlebihan sudah dihilangkan. Serta, semua huruf kapital sudah diubah menjadi huruf kecil. Setelah itu, url http yang berada di belakang setiap data teks sudah dihapus.

Menentukan panjang kalimat terpanjang pada data clean_teks

In [ ]:
max_sentence = df['clean_text'].str.split().str.len().max()
max_sentence

54

Melakukan train test split dengan bobot training: 80% dan testing: 20%

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], train_size = 0.8, random_state = 42, stratify = df['label'])

In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Melakukan tokenisasi pada data-data variabel x_train

In [ ]:
word_token = [word_tokenize(i) for i in x_train]
word_token

[['its',
  'ecb',
  'rate',
  'decision',
  'day',
  'heres',
  'what',
  'to',
  'expect',
  'via',
  'weberalexander',
  'carolynnlook'],
 ['twitter',
  'users',
  'were',
  'quick',
  'to',
  'spot',
  'liz',
  'truss',
  'seemingly',
  'recreating',
  'an',
  'outfit',
  'of',
  'margaret',
  'thatchers',
  'for',
  'her',
  'appearance',
  'at',
  'channel',
  's',
  'tory',
  'leadership',
  'debate'],
 ['jetblue',
  'announces',
  'webcast',
  'of',
  'second',
  'quarter',
  'earnings',
  'conference',
  'call'],
 ['calm',
  'calmaine',
  'foods',
  'stock',
  'ticks',
  'higher',
  'on',
  'record',
  'net',
  'income',
  'pricing',
  'power'],
 ['tower',
  'semiconductor',
  'and',
  'cadence',
  'expand',
  'collaboration',
  'to',
  'accelerate',
  'automotive',
  'ic',
  'development'],
 ['us',
  'stocks',
  'fell',
  'as',
  'earnings',
  'season',
  'is',
  'set',
  'to',
  'kick',
  'off',
  'this',
  'week',
  'and',
  'investors',
  'worried',
  'about',
  'the',
  'i

Melakukan filtering pada data-data train

In [ ]:
stopwords = set(stopwords.words('english'))

filtered_tokens_train_vec = [[word for word in sublist if word not in stopwords] for sublist in word_token]

filtered_tokens_train = [" ".join([word for word in sublist if word not in stopwords]) for sublist in word_token]
print(filtered_tokens_train[0])
print(filtered_tokens_train[1])

ecb rate decision day heres expect via weberalexander carolynnlook
twitter users quick spot liz truss seemingly recreating outfit margaret thatchers appearance channel tory leadership debate


filtered_tokens_train_vec merupakan daftar dimana setiap sublist berisi kata-kata yang tidak ada dalam daftar stopwords. filtered_tokens_train_vec digunakan untuk vectorization, sedangkan filtered_tokens_train digunakan pada saat

**Vectorization using Skipgram**

In [ ]:
import gensim
from gensim.models import Word2Vec
model_skipgram = gensim.models.Word2Vec(filtered_tokens_train_vec, min_count = 1, vector_size = 128, window = 3, sg = 1)

In [ ]:
vocabulary_skipgram = model_skipgram.wv.index_to_key
word_vec_dict = {}
for word in vocabulary_skipgram:
    word_vec_dict[word] = model_skipgram.wv.get_vector(word)
print("The no of key-value pairs : ", len(word_vec_dict))

The no of key-value pairs :  20925


In [ ]:
word_vec_dict

{'us': array([-0.01134192, -0.26190963,  0.19513354,  0.31016564,  0.3256084 ,
        -0.03078326,  0.16308735, -0.25199082, -0.1210383 ,  0.04482784,
         0.19204125,  0.10769692, -0.22064888, -0.2368869 ,  0.03104486,
         0.5042344 , -0.10275149,  0.19135651, -0.43058562,  0.41207325,
         0.10177853,  0.10231645, -0.285802  , -0.6383095 , -0.34438604,
         0.15034996, -0.31859443,  0.1081462 , -0.05119259, -0.06604039,
        -0.24879955, -0.14263368, -0.14414331,  0.2089625 , -0.07418156,
         0.21017054,  0.42797104, -0.15114743,  0.02628859,  0.02186668,
        -0.20084193,  0.03129466, -0.08484887, -0.04454075,  0.20157728,
        -0.16556332, -0.2502727 , -0.31831384, -0.18947612,  0.17448269,
         0.03455917,  0.16110978,  0.06916311,  0.02681441, -0.02342582,
        -0.17547178,  0.44758046, -0.158863  , -0.25963548,  0.3009641 ,
        -0.20585349,  0.06173736,  0.1189938 , -0.1481949 ,  0.53102964,
        -0.00602638,  0.26819313, -0.01043624

In [ ]:
from keras.preprocessing.text import Tokenizer

token = Tokenizer()

max_sentence_len = max_sentence
embed_dim = 128

Menyesuaikan tokenizer dengan teks pada filtered_tokens_train, lalu diubah menjadi sequence disimpan kedalam encd_rev. Variabel berupa daftar dimana setiap sublist-nya adalah teks-teks yang difilter, ditokenisasi, dan diubah ke sequence.

In [ ]:
token.fit_on_texts(filtered_tokens_train)
encd_rev = token.texts_to_sequences(filtered_tokens_train)
encd_rev

[[191, 31, 694, 83, 142, 497, 42, 5071, 5072],
 [148,
  1103,
  2259,
  1156,
  729,
  695,
  7745,
  11146,
  6121,
  5073,
  6122,
  6123,
  1396,
  775,
  192,
  1530],
 [3848, 13, 710, 18, 17, 3, 97, 26],
 [2410, 4366, 2127, 24, 5074, 74, 132, 255, 327, 776, 133],
 [2128, 745, 5075, 433, 1235, 1286, 1397, 6124, 553],
 [1, 4, 358, 3, 536, 106, 2622, 41, 35, 1999, 568, 6, 337, 660],
 [111, 94, 11, 155, 48, 180, 23, 6, 6125],
 [7746, 6126, 583, 3106, 11147, 2129, 338, 5076],
 [6127, 46, 965, 7747, 368, 163, 2260, 11148],
 [300, 160, 505, 108, 2130, 1287, 40, 7748],
 [64,
  676,
  2131,
  2261,
  81,
  3107,
  43,
  2262,
  2000,
  48,
  1897,
  74,
  23,
  463,
  316,
  256,
  119,
  711,
  98,
  108,
  569,
  104],
 [7749,
  1,
  11149,
  4367,
  166,
  4,
  2,
  464,
  79,
  143,
  3849,
  166,
  4,
  2,
  79,
  143,
  1700,
  166,
  4,
  2,
  79,
  143,
  5077,
  166,
  4,
  2,
  79,
  143],
 [570,
  139,
  4368,
  746,
  604,
  145,
  143,
  570,
  139,
  4368,
  746,
  481,
  145

Mendefinisikan ukuran vocabulary yang berupa panjang dari word_index pada tokenizer ditambah dengan 1.

In [ ]:
vocab_size = len(token.word_index) + 1

In [ ]:
print(vocab_size)

20918


Menghitung embedding matrix

In [ ]:
import numpy as np

embed_matrix = np.zeros(shape = (vocab_size, embed_dim))
for word, i in token.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

Melakukan padding pada teks sequence



In [ ]:
from keras.preprocessing.sequence import pad_sequences

pad_rev = pad_sequences(encd_rev, maxlen = max_sentence_len, padding = 'post')
pad_rev

array([[  191,    31,   694, ...,     0,     0,     0],
       [  148,  1103,  2259, ...,     0,     0,     0],
       [ 3848,    13,   710, ...,     0,     0,     0],
       ...,
       [  715,  1773,  2343, ...,     0,     0,     0],
       [ 3764,  4351,   381, ...,     0,     0,     0],
       [  969, 20917,   130, ...,     0,     0,     0]], dtype=int32)

In [ ]:
pad_rev.shape

(13592, 54)

**Classification**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.initializers import Constant
from keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Flatten

In [ ]:
model = Sequential([
        Embedding(vocab_size, embed_dim, embeddings_initializer = Constant(embed_matrix), input_length = max_sentence_len, trainable = False),
        LSTM(128, input_shape = (None, 28), return_sequences = True),
        Dropout(0.2),
        Flatten(),
        Dense(20)
])

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 128)           2677504   
                                                                 
 lstm (LSTM)                 (None, 54, 128)           131584    
                                                                 
 dropout (Dropout)           (None, 54, 128)           0         
                                                                 
 flatten (Flatten)           (None, 6912)              0         
                                                                 
 dense (Dense)               (None, 20)                138260    
                                                                 
Total params: 2947348 (11.24 MB)
Trainable params: 269844 (1.03 MB)
Non-trainable params: 2677504 (10.21 MB)
_________________________________________________________________
None


In [ ]:
model.compile(optimizer = "adam", loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ["accuracy"])

In [ ]:
from keras.callbacks import EarlyStopping

ES = EarlyStopping(monitor = 'loss', patience = 4, restore_best_weights = True)

model.fit(pad_rev, y_train, batch_size = 64, epochs = 50, callbacks = [ES])

Epoch 1/50
213/213 [==============================] - 28s 117ms/step - loss: 1.6571 - accuracy: 0.4928
Epoch 2/50
213/213 [==============================] - 27s 128ms/step - loss: 1.3280 - accuracy: 0.5864
Epoch 3/50
213/213 [==============================] - 29s 135ms/step - loss: 1.2278 - accuracy: 0.6120
Epoch 4/50
213/213 [==============================] - 25s 116ms/step - loss: 1.1706 - accuracy: 0.6268
Epoch 5/50
213/213 [==============================] - 27s 128ms/step - loss: 1.1215 - accuracy: 0.6444
Epoch 6/50
213/213 [==============================] - 27s 129ms/step - loss: 1.0718 - accuracy: 0.6580
Epoch 7/50
213/213 [==============================] - 25s 118ms/step - loss: 1.0234 - accuracy: 0.6714
Epoch 8/50
213/213 [==============================] - 28s 133ms/step - loss: 0.9809 - accuracy: 0.6843
Epoch 9/50
213/213 [==============================] - 27s 126ms/step - loss: 0.9454 - accuracy: 0.6957
Epoch 10/50
213/213 [==============================] - 25s 118ms/step - l

**Testing**

In [ ]:
word_token_test = [word_tokenize(i) for i in x_test]

Melakukan filtering pada data-data testing

In [ ]:
filtered_tokens_test = [[word for word in sublist if word not in stopwords] for sublist in word_token_test]
print(filtered_tokens_test[0])

['nonprofit', 'organization', 'pursuit', 'sold', 'million', 'bond', 'finance', 'job', 'training', 'program', 'needs', 'participants', 'pay', 'debt', 'future', 'earnings']


In [ ]:
encd_rev_test = token.texts_to_sequences(filtered_tokens_test)
encd_rev_test

[[4274,
  2401,
  4670,
  653,
  70,
  277,
  14,
  603,
  2969,
  367,
  532,
  4275,
  208,
  213,
  341,
  3],
 [213,
  2123,
  357,
  2737,
  241,
  899,
  7,
  621,
  1127,
  2554,
  3053,
  156,
  1312,
  212,
  72,
  1139,
  49,
  6509,
  50,
  4845,
  284,
  101],
 [851, 10, 904, 3, 5444, 7101, 538, 15704, 10, 4, 9],
 [86, 4, 48, 949, 164, 116, 267, 43, 705, 3816, 84, 72, 964, 2086],
 [5959, 119, 10762, 400, 1768, 81, 3943, 65, 550],
 [7248, 7892, 198, 209, 2551, 1165, 13038, 1027],
 [906, 6, 1836, 39, 906, 1365, 23, 156, 164, 7058, 16, 16, 6082, 446, 11],
 [127, 819, 667, 360, 17246, 1672, 897, 33, 418, 1034, 17247],
 [9794, 1887, 1962, 456, 564, 1289, 5802, 10, 460, 18815, 1962],
 [6, 2206, 2748, 1073, 172, 4457],
 [27,
  187,
  23,
  257,
  302,
  379,
  263,
  355,
  2,
  692,
  2039,
  1815,
  90,
  747,
  62,
  1674,
  8268,
  479,
  12931,
  119],
 [27, 1956, 265, 999, 874, 377, 1111, 373, 2482, 3899, 835, 499, 98, 594],
 [2184, 2092, 13, 1113, 129, 576, 407, 8342, 1912,

Teks-teks tokenizer pada filtered_tokens_test lalu diubah menjadi sequence disimpan kedalam encd_rev. Variabel berupa daftar dimana setiap sublist-nya adalah teks-teks yang difilter dan diubah ke sequence.

Melakukan padding pada teks sequence

In [ ]:
from keras.preprocessing.sequence import pad_sequences

pad_rev_test = pad_sequences(encd_rev_test, maxlen = max_sentence_len, padding = 'post')
pad_rev_test

array([[ 4274,  2401,  4670, ...,     0,     0,     0],
       [  213,  2123,   357, ...,     0,     0,     0],
       [  851,    10,   904, ...,     0,     0,     0],
       ...,
       [20754,  1888,   669, ...,     0,     0,     0],
       [  245,    13,     2, ...,     0,     0,     0],
       [  252,   584,   896, ...,     0,     0,     0]], dtype=int32)

In [ ]:
pad_rev_test.shape

(3398, 54)

Melakukan prediksi model terhadap data testing

In [ ]:
predict = model.predict(pad_rev_test)
y_pred = np.argmax(predict, axis = 1)
y_pred

107/107 [==============================] - 4s 32ms/step


array([2, 3, 5, ..., 9, 2, 7])

Menampilkan classification report, nilai accuracy, precision, recall, dan F1-score

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print('Classification Report\n')
print(classification_report(y_test, y_pred))

Classification Report

              precision    recall  f1-score   support

           0       0.39      0.24      0.29        51
           1       0.75      0.73      0.74       167
           2       0.69      0.70      0.69       709
           3       0.41      0.52      0.46        64
           4       0.95      0.88      0.91        72
           5       0.90      0.96      0.93       198
           6       0.77      0.76      0.77       105
           7       0.83      0.75      0.79       125
           8       0.67      0.48      0.56        33
           9       0.57      0.54      0.56       311
          10       0.88      0.50      0.64        14
          11       0.25      0.11      0.15         9
          12       0.71      0.62      0.66        97
          13       0.43      0.31      0.36        94
          14       0.64      0.75      0.69       364
          15       0.71      0.74      0.73       100
          16       0.75      0.80      0.77       197
    

In [ ]:
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred, average = 'micro'))
print("Recall: ", recall_score(y_test, y_pred, average = 'micro'))
print("F1-score: ", f1_score(y_test, y_pred, average = 'micro'))

Accuracy:  0.6980576809888169
Precision:  0.6980576809888169
Recall:  0.6980576809888169
F1-score:  0.6980576809888169


Dari hasil accuracy, precision, recall, dan F1-score bisa dilihat bahwa nilainya sama-sama 0.6981 atau sekitar 0.7 (70%). Serta pada training nilai akurasi yang didapatkan adalah 0.9679 (97%). Ini berarti model telah melakukan kinerja yang baik dalam melakukan klasifikasi.

**Penjelasan hyperparameter tuning:**

Pada saat melakukan hyperparameter tuning pada model arsitektur LSTM, saya menggunakan LSTM dengan unit 128, input_shape = (None, 28), dan melakukan return_sequence dengan nilai True. Serta, saya menambahkan layer Dropout dengan bobot 0.2 untuk mengurangi overfitting. Serta, Flatten untuk mengubah atau reshape dimensi dari output menjadi 1 dimensi. Dense(20) menandakan bahwa dalam dataset terdapat 20 label.

Pada saat training model, dalam compile model optimizer saya gunakan adam dan loss-nya adalah keras.losses.SparseCategoricalCrossentropy. Epoch yang digunakan adalah sebanyak 50 dan batch_size nya sebesar 64. Hal ini bertujuan agar model dapat melakukan learning dengan lebih banyak dengan epoch yang banyak. Early stopping saya gunakan dengan nilai patience = 4 untuk mengetahui pada epoch berapa nilai accuracy terbaik dihasilkan dan supaya model pada saat dilakukan training dapat menghasilkan nilai accuracy yang paling bagus.